In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import json

import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.interpolate
import scipy.signal

import pymedphys
import pymedphys._wlutz.bbpredict
# import pymedphys._wlutz.pylinac
# import pymedphys._wlutz.iview
# import pymedphys._wlutz.imginterp
# import pymedphys._wlutz.findfield
# import pymedphys._wlutz.findbb
# import pymedphys._wlutz.reporting

In [ ]:
penumbra = 2
edge_lengths = [20, 24]
initial_rotation = 0
bb_diameter = 8

pd.set_option("display.max_rows", 101)

In [ ]:
frame_paths_list = pymedphys.zip_data_paths("wlutz_arc_session.zip", check_hashes=False)

frame = [path.stem.split('_')[1] for path in frame_paths_list]
timestamps = [path.parent.stem for path in frame_paths_list]
directions = [path.parent.parent.stem for path in frame_paths_list]
beams = [path.parent.parent.parent.stem for path in frame_paths_list]

keys = list(zip(beams, directions, timestamps, frame))

image_paths = {
    key: path for key, path in zip(keys, frame_paths_list)
}

In [ ]:
key_map = {
    key: '-'.join(key) for key in keys
}

inv_key_map = {
    item: key for key, item in key_map.items()
}

In [ ]:
with open('session_cache.json', 'r') as a_file:
    data_string_keys = json.load(a_file)

data = {
    inv_key_map[key]: item for key, item in data_string_keys.items()
}

In [ ]:
movie_keys = list({
    key[0:3] for key in keys
})
movie_data_dicts = {
    movie_key: {
        int(key[3]): item for key, item in data.items()
        if key[0:3] == movie_key
    }
    for movie_key in movie_keys
}

movie_data_dicts[movie_keys[0]]

In [ ]:
movie_data = {
    movie_key: [item[frame_key] for frame_key in sorted(item.keys())]
    for movie_key, item in movie_data_dicts.items()
}

# movie_data

In [ ]:
movie_keys

In [ ]:
def extract_data(keys, data, lookup_func):
    result = {}

    for key in keys:
        result[key] = []
        for item in data[key]:
            try:
                result[key].append(lookup_func(item))
            except KeyError:
                result[key].append(np.nan)

        result[key] = np.array(result[key])
        
    return result


In [ ]:
pymedphys_field_rotations = extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['field_rotation'])

In [ ]:
pymedphys_field_rotations

In [ ]:
# pymedphys_field_rotations = {}

# for key in movie_keys:
#     pymedphys_field_rotations[key] = []
#     for item in movie_data[key]:
#         try:
#             pymedphys_field_rotations[key].append(item['pymedphys']['field_rotation'])
#         except KeyError:
#             pass
        
#     pymedphys_field_rotations[key] = np.array(rotations[key])

In [ ]:
def determine_gantry_angle(direction_key, rotation):
    not_nan = np.invert(np.isnan(rotation))
    nan_removed_rotation = rotation[not_nan]
    
    if direction_key == 'clockwise':
        diff = np.diff(np.concatenate([[-180], nan_removed_rotation]))
        diff[diff > 0] = diff[diff > 0] - 180

        gantry = -180 - np.cumsum(diff * 2)
    elif direction_key == 'counter-clockwise':
        diff = np.diff(np.concatenate([[0], nan_removed_rotation]))
        diff[diff < 0] = diff[diff < 0] + 180

        gantry = 180 - np.cumsum(diff * 2)
    else:
        raise ValueError("Expected one of 'clockwise' or 'counter-clockwise'")
        
    gantry_with_nans = np.ones_like(rotation) * np.nan
    out_of_bounds = np.logical_or(gantry < -180, gantry > 180)
    gantry[out_of_bounds] = np.nan
    gantry_with_nans[not_nan] = gantry
        
    return gantry_with_nans

In [ ]:
movie_keys[2]

In [ ]:
pymedphys_field_rotations[movie_keys[2]]

In [ ]:
gantry_angles = {}

for key in movie_keys:
    direction_key = key[1]
    rotation = pymedphys_field_rotations[key]
    
    gantry_angles[key] = determine_gantry_angle(direction_key, rotation)
    
gantry_angles

In [ ]:
columns=[
    'gantry_angle', 'field_x', 'field_y', 'bb_x', 'bb_y'
]

In [ ]:
prep_for_dataframe = [
    gantry_angles,
    extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['field_centre'][0]),
    extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['field_centre'][1]),
    extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['bb_centre'][0]),
    extract_data(movie_keys, movie_data, lambda x: x['pymedphys']['bb_centre'][1]),
]

In [ ]:
dataframes = {}

for key in movie_keys:
    dataframe = pd.DataFrame(
        columns=columns,
        data=np.vstack([
            item[key] for item in prep_for_dataframe
        ]).T
    )
    
    dataframes[key] = dataframe

In [ ]:
for key in movie_keys:
    print(key)
    IPython.display.display(dataframes[key])

In [ ]:



# def create_bb_predictor(bb_x, bb_y, gantries, directions):
#     bb_coords_keys = ['x', 'y']
#     direction_options = np.unique(directions)
#     prediction_functions = {}
    
#     for bb_coords_key, bb_coords in zip(bb_coords_keys, [bb_x, bb_y]):
#         for current_direction in direction_options:
#             interps = [
#                 scipy.interpolate.interp1d(gantry, bb_coord, bounds_error=False, fill_value='extrapolate')
#                 for gantry, bb_coord, direction in zip(gantries, bb_coords, directions) 
#                 if direction == current_direction
#             ]
            
#             def prediction_func(gantry, tol=20):
#                 results = []

#                 for interp in interps:
#                     results.append(interp(gantry))

#                 min_val = np.nanmin(results, axis=0)
#                 max_val = np.nanmax(results, axis=0)
#                 result = np.nanmean(results, axis=0)
                
#                 out_of_tol = np.logical_or(
#                     max_val - result > tol,
#                     result - min_val > tol
#                 )
#                 result[out_of_tol] = np.nan

#                 return result
            
#             prediction_functions[(bb_coords_key, current_direction)] = prediction_func
    
#     def predict_bb(gantry, direction, tol=0.1):
#         results = []
#         for bb_coords_key in bb_coords_keys:
#             results.append(prediction_functions[(bb_coords_key, direction)](gantry, tol=tol))
            
#         return results
    
    
#     return predict_bb
            

bb_x = [
    dataframes[key]['bb_x'] for key in movie_keys
]
bb_y = [
    dataframes[key]['bb_y'] for key in movie_keys
]
gantry = [
    gantry_angles[key] for key in movie_keys
]
direction = [key[1] for key in movie_keys]

predict_bb = pymedphys._wlutz.bbpredict.create_bb_predictor(
    bb_x, bb_y, gantry, direction)

predict_bb([0, 2], 'clockwise')


In [ ]:
gantry_i = np.linspace(-180, 180, 91)
predict_bb(gantry_i, 'clockwise')

In [ ]:
# np.unique?

In [ ]:




# interp_bb_x = [
#     scipy.interpolate.interp1d(g, x, bounds_error=False, fill_value='extrapolate')
#     for g, x in zip(gantry, bb_x)
# ]

# def get_avg_bb_x(gantry, range_tol=0.2):
#     results = []
    
#     for interp in interp_bb_x:
#         results.append(interp(gantry))
        
#     min_val = np.nanmin(results, axis=0)
#     max_val = np.nanmax(results, axis=0)
#     out_of_tol = max_val - min_val > range_tol
    
#     result = (np.nanmin(results, axis=0) + np.nanmax(results, axis=0))/2
#     result[out_of_tol] = np.nan
    
#     return result


# interp_bb_y = [
#     scipy.interpolate.interp1d(g, y, bounds_error=False, fill_value='extrapolate')
#     for g, y in zip(gantry, bb_y)
# ]

# def get_avg_bb_y(gantry, range_tol=0.2):
#     results = []
    
#     for interp in interp_bb_y:
#         results.append(interp(gantry))
    
#     min_val = np.nanmin(results, axis=0)
#     max_val = np.nanmax(results, axis=0)
#     out_of_tol = max_val - min_val > range_tol
    
#     result = (np.nanmin(results, axis=0) + np.nanmax(results, axis=0))/2
#     result[out_of_tol] = np.nan
    
#     return result

# get_avg_bb_y([0, 2])

In [ ]:
gantry_i = np.linspace(-180, 180, 91)

In [ ]:
plt.figure(figsize=(12,10))

for g, x, key in zip(gantry, bb_x, movie_keys):
    if key[1] == 'clockwise':
        prop = '-'
    else:
        prop = '--'
        
    plt.plot(g, x, prop, alpha=0.5, label=key[0:2])
    
plt.plot(gantry_i, predict_bb(gantry_i, 'clockwise')[0], 'k')
plt.plot(gantry_i, predict_bb(gantry_i, 'counter-clockwise')[0], 'k--')
plt.legend()

In [ ]:
predict_bb(gantry_i, 'counter-clockwise')

In [ ]:
plt.figure(figsize=(12,10))

for g, y, key in zip(gantry, bb_y, movie_keys):
    if key[1] == 'clockwise':
        prop = '-'
    else:
        prop = '--'
        
    plt.plot(g, y, prop, alpha=0.5, label=key[0:2])
    
plt.plot(gantry_i, predict_bb(gantry_i, 'clockwise')[1], 'k')
plt.plot(gantry_i, predict_bb(gantry_i, 'counter-clockwise')[1], 'k--')
plt.legend()

In [ ]:
for key in movie_keys:
    bb_x = dataframes[key]['bb_x']
    bb_y = dataframes[key]['bb_y']
    gantry = dataframes[key]['gantry_angle']
    direction = key[1]
    
    isnan = np.isnan(bb_x)
    assert np.all(isnan == np.isnan(bb_y))
    
    bb_x_prediction, bb_y_prediction = predict_bb(gantry[isnan], direction)
    
    bb_x[isnan] = bb_x_prediction
    bb_y[isnan] = bb_y_prediction
    
    dataframes[key]['bb_x'] = bb_x
    dataframes[key]['bb_y'] = bb_y

In [ ]:
for key in movie_keys:
    print(key)
    IPython.display.display(dataframes[key])

In [ ]:
analysis_dataframes = {}

for key in movie_keys:
    print(key)
    analysis_dataframe = pd.DataFrame(
        columns=['Gantry Angle', 'Field - BB x', 'Field - BB y'],
        data=np.vstack([
            dataframes[key]['gantry_angle'], 
            dataframes[key]['field_x'] - dataframes[key]['bb_x'],
            dataframes[key]['field_y'] - dataframes[key]['bb_y']
        ]).T
    )
    
    IPython.display.display(analysis_dataframe)
    
    analysis_dataframes[key] = analysis_dataframe

In [ ]:
# plt.figure(figsize=(12,10))

# for key in movie_keys:
#     plt.plot(analysis_dataframes[key]['Gantry Angle'], analysis_dataframes[key]['Field - BB x'], label=key[0:2])
    
# plt.legend()

In [ ]:
# plt.figure(figsize=(12,10))

# for key in movie_keys:
#     plt.plot(analysis_dataframes[key]['Gantry Angle'], analysis_dataframes[key]['Field - BB y'], label=key[0:2])
    
# plt.legend()

In [ ]:
def plot_enery_axis(energy, axis):
    plt.figure(figsize=(12,10))

    for key in movie_keys:
        if key[0] == energy:
            if key[1] == 'clockwise':
                prop = '-'
            else:
                prop = '--'

            plt.plot(
                analysis_dataframes[key]['Gantry Angle'], 
                analysis_dataframes[key][f'Field - BB {axis}'], 
                prop, label=key[0:2], alpha=0.8)
            
    x = np.linspace(-180, 180)

    if axis == 'y':
        plt.plot(x, 0.6*np.cos(x*np.pi/180), 'k', label='"Ideal"')
        plt.plot(x, 0.6*np.cos(x*np.pi/180)-0.5, 'r', label='0.5 mm "bounds"', alpha=0.2)
        plt.plot(x, 0.6*np.cos(x*np.pi/180)+0.5, 'r', alpha=0.2)
        
    elif axis == 'x':
        plt.plot(x, np.zeros_like(x), 'k', label='"Ideal"')
        plt.plot(x, np.zeros_like(x)-0.5, 'r', label='0.5 mm "bounds"', alpha=0.2)
        plt.plot(x, np.zeros_like(x)+0.5, 'r', alpha=0.2)

    plt.legend()

In [ ]:
plot_enery_axis('06MV', 'x')

In [ ]:
plot_enery_axis('06MV', 'y')

In [ ]:
plot_enery_axis('10MV', 'x')

In [ ]:
plot_enery_axis('10MV', 'y')

In [ ]:
plot_enery_axis('06FFF', 'x')

In [ ]:
plot_enery_axis('06FFF', 'y')

In [ ]:
plot_enery_axis('10FFF', 'x')

In [ ]:
plot_enery_axis('10FFF', 'y')